# AWS Inferentia inference on Amazon EC2 Inf1 instance
This example demonstrates AWS Inferentia inference with TensorFlow and AWS Neuron SDK compiler and runtime

This example was tested on Amazon EC2 `inf1.xlarge` the following AWS Deep Learning AMI: 
`Deep Learning AMI (Ubuntu 18.04) Version 35.0`

Run this notebook using the following conda environment:
`aws_neuron_tensorflow_p36`

Prepare your imagenet validation TFRecord files using the following helper script:
https://github.com/tensorflow/models/blob/archive/research/inception/inception/data/download_and_preprocess_imagenet.sh

Save it to `/home/ubuntu/datasets/` or update the dataset location in the `get_dataset()` function

In [1]:
import os
import time
import shutil
import json
import requests
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.neuron as tfn
import tensorflow.compat.v1.keras as keras
# from tensorflow.keras.applications import ( 
#     xception,
#     vgg16,
#     vgg19,
#     resnet,
#     resnet50,
#     resnet_v2,
#     inception_v3,
#     inception_resnet_v2,
#     mobilenet,
#     densenet,
#     nasnet,
#     mobilenet_v2
# )
# # from keras import backend as K
# from tensorflow.keras.preprocessing import image
# # from concurrent import futures
# from itertools import compress


# models = {
# #     'xception':xception,
#     'vgg16':vgg16,
# #     'vgg19':vgg19,
# #     'resnet50':resnet50,
# #     'resnet101':resnet,
# #     'resnet152':resnet,
# #     'resnet50_v2':resnet_v2,
# #     'resnet101_v2':resnet_v2,
# #     'resnet152_v2':resnet_v2,
# #     'resnext50':resnext,
# #     'resnext101':resnext,
# #     'inception_v3':inception_v3,
# #     'inception_resnet_v2':inception_resnet_v2,
# #     'mobilenet':mobilenet,
# #     'densenet121':densenet,
# #     'densenet169':densenet,
# #     'densenet201':densenet,
# #     'nasnet':nasnet,
# #     'nasnet':nasnet,
# #     'mobilenet_v2':mobilenet_v2
# }

# models_detail = {
# #     'xception':xception.Xception(weights='imagenet'),
#     'vgg16':vgg16.VGG16(weights='imagenet'),
# #     'vgg19':vgg19.VGG19(weights='imagenet'),
# #     'resnet50':resnet50.ResNet50(weights='imagenet'),
# #     'resnet101':resnet.ResNet101(weights='imagenet'),
# #     'resnet152':resnet.ResNet152(weights='imagenet'),
# #     'resnet50_v2':resnet_v2.ResNet50V2(weights='imagenet'),
# #     'resnet101_v2':resnet_v2.ResNet101V2(weights='imagenet'),
# #     'resnet152_v2':resnet_v2.ResNet152V2(weights='imagenet'),
# #     'resnext50':resnext.ResNeXt50(weights='imagenet'),
# #     'resnext101':resnext.ResNeXt101(weights='imagenet'),
# #     'inception_v3':inception_v3.InceptionV3(weights='imagenet'),
# #     'inception_resnet_v2':inception_resnet_v2.InceptionResNetV2(weights='imagenet'),
# #     'mobilenet':mobilenet.MobileNet(weights='imagenet'),
# #     'densenet121':densenet.DenseNet121(weights='imagenet'),
# #     'densenet169':densenet.DenseNet169(weights='imagenet'),
# #     'densenet201':densenet.DenseNet201(weights='imagenet'),
# #     'nasnet':nasnet.NASNetLarge(weights='imagenet'),
# #     'nasnet':nasnet.NASNetMobile(weights='imagenet'),
# #     'mobilenet_v2':mobilenet_v2.MobileNetV2(weights='imagenet')
# }


print('test')

2022-11-17 02:02:26.586645: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


test


2022-11-17 02:02:27.506972: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-11-17 02:02:27.608286: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-17 02:02:27.608323: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-31-61-116.us-west-2.compute.internal): /proc/driver/nvidia/version does not exist


In [2]:
# model_type = 'vgg16'

# # https://github.com/tensorflow/tensorflow/issues/29931
# temp = tf.zeros([8, 224, 224, 3])
# _ = models[model_type].preprocess_input(temp)

# print('test2')

### Resnet50 FP32 saved model

In [3]:
# Export SavedModel

model_name = 'models/vgg16_conv64'
saved_model_dir = f'{model_name}'
# shutil.rmtree(saved_model_dir, ignore_errors=True)

# model = models_detail[model_type]

# model.save(saved_model_dir)


from tensorflow.keras.models import load_model
model = load_model(saved_model_dir, compile=True)

model.summary()

2022-11-17 02:02:27.683316: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
Total params: 1,792
Trainable params: 1,792
Non-trainable params: 0
_________________________________________________________________


### Compile models with different batch sizes and cores

In [4]:
def compile_inf1_model(saved_model_dir, inf1_model_dir, batch_size=1, num_cores=1, use_static_weights=False):
    print(f'-----------batch size: {batch_size}, num cores: {num_cores}----------')
    print('Compiling...')
    
    compiled_model_dir = f'{model_name}_batch_{batch_size}_inf1_cores_{num_cores}'
    inf1_compiled_model_dir = os.path.join(inf1_model_dir, compiled_model_dir)
    shutil.rmtree(inf1_compiled_model_dir, ignore_errors=True)

    example_input = np.zeros([batch_size,224,224,3], dtype='float32')
    !env NEURON_CC_FLAGS="--neuroncore-pipeline-cores=1"
    
    start_time = time.time()
    compiled_model = tfn.trace(model,example_input)
    compiled_res = compiled_model.save(inf1_compiled_model_dir)
    print(f'Compile time: {time.time() - start_time}')
    
    compile_success = False
#     perc_on_inf = compiled_res['OnNeuronRatio'] * 100
#     if perc_on_inf > 50:
#         compile_success = True
            
    print(inf1_compiled_model_dir)
    print(compiled_res)
    print('----------- Done! ----------- \n')
    
    return compile_success

In [5]:
inf1_model_dir = f'{model_name}_inf1_saved_models'
saved_model_dir = f'{model_name}'


# testing batch size
batch_list = [1]
num_of_cores = [1]
for batch in batch_list:
    for core in num_of_cores:
        print('batch size:', batch,'core nums', core,'compile start')
        compile_inf1_model(saved_model_dir, inf1_model_dir, batch_size=batch, num_cores=core)


batch size: 1 core nums 1 compile start
-----------batch size: 1, num cores: 1----------
Compiling...
XDG_SESSION_ID=1
HOSTNAME=ip-172-31-61-116.us-west-2.compute.internal
SHELL=/bin/bash
TERM=xterm-color
CLICOLOR=1
HISTSIZE=1000
SSH_CLIENT=203.246.112.118 54595 22
PYDEVD_USE_FRAME_EVAL=NO
SSH_TTY=/dev/pts/0
USER=ec2-user
LS_COLORS=rs=0:di=38;5;27:ln=38;5;51:mh=44;38;5;15:pi=40;38;5;11:so=38;5;13:do=38;5;5:bd=48;5;232;38;5;11:cd=48;5;232;38;5;3:or=48;5;232;38;5;9:mi=05;48;5;232;38;5;15:su=48;5;196;38;5;15:sg=48;5;11;38;5;16:ca=48;5;196;38;5;226:tw=48;5;10;38;5;16:ow=48;5;10;38;5;21:st=48;5;21;38;5;15:ex=38;5;34:*.tar=38;5;9:*.tgz=38;5;9:*.arc=38;5;9:*.arj=38;5;9:*.taz=38;5;9:*.lha=38;5;9:*.lz4=38;5;9:*.lzh=38;5;9:*.lzma=38;5;9:*.tlz=38;5;9:*.txz=38;5;9:*.tzo=38;5;9:*.t7z=38;5;9:*.zip=38;5;9:*.z=38;5;9:*.Z=38;5;9:*.dz=38;5;9:*.gz=38;5;9:*.lrz=38;5;9:*.lz=38;5;9:*.lzo=38;5;9:*.xz=38;5;9:*.bz2=38;5;9:*.bz=38;5;9:*.tbz=38;5;9:*.tbz2=38;5;9:*.tz=38;5;9:*.deb=38;5;9:*.rpm=38;5;9:*.jar=38;5;9

2022-11-17 02:02:28.347267: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-11-17 02:02:28.347442: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-11-17 02:02:28.348158: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2999995000 Hz
2022-11-17 02:02:28.360377: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2022-11-17 02:02:28.360494: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-11-17 02:02:28.362846: I tensorflow/neuron/grappler/convert/segment.cc:456] There are 3 ops of 2 different types in the graph that are not compiled by neuron-cc: Placeholder, NoOp, (For more information see https://awsdocs-neuron.readthedocs-hosted.com/en/latest/release-notes/neuron-cc-ops/neuron-cc-ops-tensorflow.html).
2022-11-17 02:02:28.364964: I tensorflow/core/grappler/d

INFO:tensorflow:Assets written to: models/vgg16_conv64_inf1_saved_models/models/vgg16_conv64_batch_1_inf1_cores_1/assets
Compile time: 7.50426983833313
models/vgg16_conv64_inf1_saved_models/models/vgg16_conv64_batch_1_inf1_cores_1
None
----------- Done! ----------- 

